# Talk about Framing and outlining the problem (Business problem and specifics)
# Talk about data

# Will it be delayed?

Everyone who has flown has experienced a delayed or cancelled flight. Both airlines and airports would like to improve their on-time performance and predict when a flight will be delayed or cancelled several days in advance. You are being hired to build a model that can predict if a flight will be delayed. To learn more, you must schedule a meeting with your client (me). To schedule an appointment with your client, send an event request through Google Calendar for a 15 minute meeting. Both you and your project partner must attend the meeting. Come prepared with questions to ask your client. Remember that your client is not a data scientist and you will need to explain things in a way that is easy to understand. Make sure that your communications are efficient, thought out, and not redundant as your client might get frustrated and "fire" you (this only applies to getting information from your client, this does not necessary apply to asking for help with the actual project itself - you should continuously ask questions for getting help).

For this project you must go through most all steps in the checklist. You must write responses for all items as done in the homeworks, however sometimes the item will simply be "does not apply". Keep your progress and thoughts organized in this document and use formatting as appropriate (using markdown to add headers and sub-headers for each major part). Some changes to the checklist:

* Do not do the final part (launching the product).
* Your presentation will be done as information written in this document in a dedicated section (no slides or anything like that). It should include high-level summary of your results (including what you learned about the data, the "accuracy" of your model, what features were important, etc). It should be written for your client, not your professor or teammates. It should include the best summary plots/graphics/data points.
* The models and hyperparameters you should consider during short-listing and fine-tuning will be released at a later time (dependent on how far we get over the next two weeks).
* Data retrieval must be automatic as part of the code (so it can easily be re-run and grab the latest data). Do not commit any data to the repository.
* Your submission must include a pickled final model along with this notebook.


# Notes

* gov agency tasked with improving efficiency with comercial airtraffic from consumer standpoint, 
* predict delays and errors 7 days in advanced
* data is from the 80s, subitted take off, supposed to leave, etc, on BTS (get a link) only look at 2023 and 2024
* less predictions but more accurate predictions by a 1/4
* National Air System (group that runs opperations at a single airport, makes all decisions so something they do can cause a run down, weather issues, security (TSA, bad person on deck), late arrival, cancelations,  ) broken down into minor, medium, major categories
* Things to ignore in data, diverted, international destination or arrival (only USA to USA)

In [1]:
!pip install numpy scipy pandas matplotlib scikit-learn pyarrow fastparquet seaborn 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# All of your imports here (you may need to add some)
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import pyarrow as pa
import fastparquet as fp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import glob

# Frame problem, get data, explore

# Combing all the files into one for each year


In [3]:
#Read in all the CSV files in the 2023_Data folder
file_paths = glob.glob('2023_Data/*.csv')

# Combine all the data into a single DataFrame
data_frames = [pd.read_csv(file_path) for file_path in file_paths]
combined_df = pd.concat(data_frames, ignore_index=True)

#saving the combined data to a parquet file
combined_df.to_parquet('2023_combined_data.parquet')

/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_22400/993464474.py:5: DtypeWarning: Columns (11,13,78,85,86,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frames = [pd.read_csv(file_path) for file_path in file_paths]
/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_22400/993464474.py:5: DtypeWarning: Columns (11,13,57,78,85,86,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frames = [pd.read_csv(file_path) for file_path in file_paths]
/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_22400/993464474.py:5: DtypeWarning: Columns (11,13,57,86,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frames = [pd.read_csv(file_path) for file_path in file_paths]
/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_22400/993464474.py:5: DtypeWarning: Columns (11,13,86,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  data_f

In [4]:
#Read in all the CSV files in the 2024_Data folder
file_paths = glob.glob('2024_Data/*.csv')

# Combine all the data into a single DataFrame
data_frames = [pd.read_csv(file_path) for file_path in file_paths]
combined_df = pd.concat(data_frames, ignore_index=True)

#saving the combined data to a parquet file
combined_df.to_parquet('2024_combined_data.parquet')

/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_22400/4027738665.py:5: DtypeWarning: Columns (11,13,86,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frames = [pd.read_csv(file_path) for file_path in file_paths]
/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_22400/4027738665.py:5: DtypeWarning: Columns (11,13,57,86,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frames = [pd.read_csv(file_path) for file_path in file_paths]
/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_22400/4027738665.py:5: DtypeWarning: Columns (11,13,86) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frames = [pd.read_csv(file_path) for file_path in file_paths]
/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_22400/4027738665.py:5: DtypeWarning: Columns (11,13,86,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frames = [pd.read_

In [3]:
# #dropping Diverted data

# Import the Parquet file
train = pd.read_parquet('2023_combined_data.parquet')

# List of columns to remove
columns_to_remove = [
    'Diverted',  # First, drop the main DIVERTED column
    'DivAirportLandings', 'DivReachedDest', 'DivActualElapsedTime', 
    'DivArrDelay', 'DivDistance', 'Div1Airport', 'Div1WheelsOn', 'Div1TotalGTime', 
    'Div1LongestGTime', 'Div1WheelsOff', 'Div1TailNum',
    'Div1AirportID', 'Div1AirportSeqID',
    'Div1TotalGTime', 'Div1LongestGTime', 'Div2Airport', 'Div2WheelsOn', 
    'Div2TotalGTime', 'Div2LongestGTime', 'Div2WheelsOff', 'Div2TailNum','Div2AirportID', 'Div2AirportSeqID', 'Div2TotalGTime', 'Div2LongestGTime', 
    'Div3Airport', 'Div3WheelsOn', 'Div3TotalGTime', 'Div3LongestGTime', 'Div3WheelsOff', 
    'Div3TailNum','Div3AirportID', 'Div3AirportSeqID', 'Div3TotalGTime', 'Div3LongestGTime', 
    'Div4Airport', 'Div4WheelsOn', 'Div4TotalGTime', 'Div4LongestGTime', 'Div4WheelsOff', 
    'Div4TailNum', 'Div4AirportID', 'Div4AirportSeqID', 'Div4TotalGTime', 'Div4LongestGTime', 
    'Div5Airport', 'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff', 
    'Div5TailNum', 'Div5AirportID', 'Div5AirportSeqID', 'Div5TotalGTime', 'Div5LongestGTime'
]

# Drop the specified columns
X = train.drop(columns=columns_to_remove, errors='ignore')  # Use errors='ignore' to prevent issues if columns don't exist

# Save the cleaned DataFrame to a new Parquet file
X.to_parquet('2023_cleaned_data.parquet')

# Shape and preview
print(X.shape)  # Check new shape
print(X.dtypes)  # Check data types


(7278739, 74)
Year                 int64
Quarter              int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
                    ...   
FirstDepTime       float64
TotalAddGTime      float64
LongestAddGTime    float64
Duplicate           object
Unnamed: 119       float64
Length: 74, dtype: object


In [4]:
#print the cleaned data
X.head()


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,Duplicate,Unnamed: 119
0,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,66.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,N,NaN
3,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [5]:
# #dropping Diverted data

# Import the Parquet file
train = pd.read_parquet('2024_combined_data.parquet')

# List of columns to remove
columns_to_remove = [
    'Diverted',  # First, drop the main DIVERTED column
    'DivAirportLandings', 'DivReachedDest', 'DivActualElapsedTime', 
    'DivArrDelay', 'DivDistance', 'Div1Airport', 'Div1WheelsOn', 'Div1TotalGTime', 
    'Div1LongestGTime', 'Div1WheelsOff', 'Div1TailNum',
    'Div1AirportID', 'Div1AirportSeqID',
    'Div1TotalGTime', 'Div1LongestGTime', 'Div2Airport', 'Div2WheelsOn', 
    'Div2TotalGTime', 'Div2LongestGTime', 'Div2WheelsOff', 'Div2TailNum','Div2AirportID', 'Div2AirportSeqID', 'Div2TotalGTime', 'Div2LongestGTime', 
    'Div3Airport', 'Div3WheelsOn', 'Div3TotalGTime', 'Div3LongestGTime', 'Div3WheelsOff', 
    'Div3TailNum','Div3AirportID', 'Div3AirportSeqID', 'Div3TotalGTime', 'Div3LongestGTime', 
    'Div4Airport', 'Div4WheelsOn', 'Div4TotalGTime', 'Div4LongestGTime', 'Div4WheelsOff', 
    'Div4TailNum', 'Div4AirportID', 'Div4AirportSeqID', 'Div4TotalGTime', 'Div4LongestGTime', 
    'Div5Airport', 'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff', 
    'Div5TailNum', 'Div5AirportID', 'Div5AirportSeqID', 'Div5TotalGTime', 'Div5LongestGTime'
]

# Drop the specified columns
X = train.drop(columns=columns_to_remove, errors='ignore')  # Use errors='ignore' to prevent issues if columns don't exist

# Save the cleaned DataFrame to a new Parquet file
X.to_parquet('2024_cleaned_data.parquet')

# Shape and preview
print(X.shape)  # Check new shape
print(X.dtypes)  # Check data types

(7546968, 74)
Year                 int64
Quarter              int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
                    ...   
FirstDepTime       float64
TotalAddGTime      float64
LongestAddGTime    float64
Duplicate           object
Unnamed: 119       float64
Length: 74, dtype: object


# Maybe Linear Regression

In [ ]:
#print the full cleaned data
X


# # Linear Regression Model
# Import the cleaned data
# train = pd.read_parquet('2023_cleaned_data.parquet')
# test = pd.read_parquet('2024_cleaned_data.parquet')


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,Duplicate,Unnamed: 119
0,2024,2,6,11,2,2024-06-11,NK,NK,20416,NK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2024,2,6,12,3,2024-06-12,NK,NK,20416,NK,...,23.0,0.0,568.0,0.0,0.0,NaN,NaN,NaN,N,NaN
2,2024,2,6,13,4,2024-06-13,NK,NK,20416,NK,...,0.0,0.0,263.0,0.0,0.0,NaN,NaN,NaN,N,NaN
3,2024,2,6,14,5,2024-06-14,NK,NK,20416,NK,...,0.0,0.0,10.0,0.0,10.0,NaN,NaN,NaN,N,NaN
4,2024,2,6,15,6,2024-06-15,NK,NK,20416,NK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7546963,2024,2,4,1,1,2024-04-01,UA,UA_CODESHARE,19977,UA,...,9.0,0.0,0.0,0.0,50.0,NaN,NaN,NaN,N,NaN
7546964,2024,2,4,1,1,2024-04-01,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
7546965,2024,2,4,1,1,2024-04-01,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
7546966,2024,2,4,1,1,2024-04-01,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


: 

In [ ]:
#finding data types
df = pd.read_csv('2023_Data/2023_April.csv')  # Load CSV into a DataFrame
print(df.dtypes)

/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_5007/4017789505.py:2: DtypeWarning: Columns (11,13,86,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2023_Data/2023_April.csv')  # Load CSV into a DataFrame


Year                  int64
Quarter               int64
Month                 int64
DayofMonth            int64
DayOfWeek             int64
                     ...   
Div5LongestGTime    float64
Div5WheelsOff       float64
Div5TailNum         float64
Duplicate            object
Unnamed: 119        float64
Length: 120, dtype: object


: 

In [4]:
# Read the parquet file
df = pd.read_parquet('2023_combined_data.parquet')
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,2023,3,9,24,7,2023-09-24,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [50]:
def read_csv(file_path):
    """Reads a CSV file and returns its content as a Pandas DataFrame."""
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return None


read_csv('T_ONTIME_REPORTING.csv')

/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_40410/3866758592.py:4: DtypeWarning: Columns (77,84,85,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
0,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N131EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N132EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N132EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N133EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N133EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547266,2024,1,1,31,3,1/31/2024 12:00:00 AM,YX,20452,YX,N879RW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547267,2024,1,1,31,3,1/31/2024 12:00:00 AM,YX,20452,YX,N882RW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547268,2024,1,1,31,3,1/31/2024 12:00:00 AM,YX,20452,YX,N882RW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547269,2024,1,1,31,3,1/31/2024 12:00:00 AM,YX,20452,YX,N882RW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
print(train_set.columns)

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM',
       ...
       'DIV4_WHEELS_OFF', 'DIV4_TAIL_NUM', 'DIV5_AIRPORT', 'DIV5_AIRPORT_ID',
       'DIV5_AIRPORT_SEQ_ID', 'DIV5_WHEELS_ON', 'DIV5_TOTAL_GTIME',
       'DIV5_LONGEST_GTIME', 'DIV5_WHEELS_OFF', 'DIV5_TAIL_NUM'],
      dtype='object', length=109)


In [52]:
train_set = read_csv('T_ONTIME_REPORTING.csv')
train_set.head()

/var/folders/v3/qw7hpymn01549bjw7kf8th940000gn/T/ipykernel_40410/3866758592.py:4: DtypeWarning: Columns (77,84,85,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
0,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N131EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N132EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N132EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N133EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N133EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Things to ignore in data, diverted, international destination or arrival (only USA to USA)

In [53]:
train_sample = train_set.sample(n=1000, random_state=42)

X = train_sample.copy()
y = X.pop('DIVERTED')

X.shape

X.head()


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
110938,2024,1,1,7,7,1/7/2024 12:00:00 AM,AA,19805,AA,N989NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17170,2024,1,1,1,1,1/1/2024 12:00:00 AM,YX,20452,YX,N745YX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496423,2024,1,1,29,1,1/29/2024 12:00:00 AM,9E,20363,9E,N937XJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166047,2024,1,1,10,3,1/10/2024 12:00:00 AM,B6,20409,B6,N974JT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176788,2024,1,1,10,3,1/10/2024 12:00:00 AM,WN,19393,WN,N8701Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#droping all the FIPS numbers outside of the US (1-56) 

us_fips_codes = {1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,
                21,22,23,24,25,26,27,28,29,30,31,32,33,34,
                35,36,37,38,39,40,41,42,44,45,46,47,48,49,
                50,51,53,54,55,56}

train_sample_filtered = train_sample[
    train_sample['ORIGIN_STATE_FIPS'].isin(us_fips_codes) &
    train_sample['DEST_STATE_FIPS'].isin(us_fips_codes)
]

train_sample_filtered.shape
# train_sample_filtered.head()
train_sample_filtered


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
110938,2024,1,1,7,7,1/7/2024 12:00:00 AM,AA,19805,AA,N989NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17170,2024,1,1,1,1,1/1/2024 12:00:00 AM,YX,20452,YX,N745YX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496423,2024,1,1,29,1,1/29/2024 12:00:00 AM,9E,20363,9E,N937XJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166047,2024,1,1,10,3,1/10/2024 12:00:00 AM,B6,20409,B6,N974JT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226521,2024,1,1,13,6,1/13/2024 12:00:00 AM,UA,19977,UA,N34131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351044,2024,1,1,20,6,1/20/2024 12:00:00 AM,WN,19393,WN,N7819A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302369,2024,1,1,18,4,1/18/2024 12:00:00 AM,AA,19805,AA,N452AN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392940,2024,1,1,23,2,1/23/2024 12:00:00 AM,AA,19805,AA,N825NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139856,2024,1,1,8,1,1/8/2024 12:00:00 AM,UA,19977,UA,N485UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
train_sample_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 981 entries, 110938 to 126418
Columns: 108 entries, YEAR to DIV5_TAIL_NUM
dtypes: float64(66), int64(21), object(21)
memory usage: 835.4+ KB


In [ ]:
# summay of statistics of the data
train_sample_filtered.describe()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,...,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
count,981.0,981.0,981.0,981.000000,981.000000,981.000000,981.000000,981.000000,9.810000e+02,981.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2024.0,1.0,1.0,16.260958,3.759429,19954.216106,2351.543323,12650.384302,1.265042e+06,31671.615698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,0.0,0.0,9.031871,1.996738,376.951884,1544.607681,1504.274419,1.504272e+05,1261.774841,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2024.0,1.0,1.0,1.000000,1.000000,19393.000000,9.000000,10140.000000,1.014005e+06,30140.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2024.0,1.0,1.0,8.000000,2.000000,19790.000000,1166.000000,11292.000000,1.129202e+06,30559.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2024.0,1.0,1.0,16.000000,4.000000,19930.000000,2089.000000,12889.000000,1.288904e+06,31453.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2024.0,1.0,1.0,24.000000,5.000000,20363.000000,3460.000000,13930.000000,1.393008e+06,32467.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2024.0,1.0,1.0,31.000000,7.000000,20452.000000,5829.000000,15624.000000,1.562404e+06,35412.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# plt.figure(figsize=(10, 6))
# sns.countplot(x='DIVERTED', data=train_sample_filtered)
# plt.title('Distribution of DIVERTED Flights')
# plt.xlabel('DIVERTED')
# plt.ylabel('Count')
# plt.show()

# Correlation matrix

correlation_matrix = train_sample_filtered.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


ValueError: could not convert string to float: '1/7/2024 12:00:00 AM'